---
title: "VRPCD with information exchange"
author: "FA"
date: 2021-01-07
tags: ["VRPCD", "realtime", "information exchange"]
categories: "research"
---

In [49]:
#*****************************************
# Necessary Package
#*****************************************
using Random
using JuMP
using LinearAlgebra
using GLPK

In [50]:
#*****************************************
# Network Property
#*****************************************
# P: set of pickup nodes
P = [2,3]

# D: set of delivery nodes
D = [5,6]

# O: cross-dock (assume single cross dock in the network)
O = [4]

# O1: dummy cross-dock
O_pickup = [1]
O_delivery = [7]

# pickup nodes
N_pickup = vcat(O_pickup, P, O)

# delivery nodes
N_delivery = vcat(O, D, O_delivery)

# all nodes
N = vcat(O_pickup, P, O, D, O_delivery)

# M: set of available vehicles
M = ["v1", "v2"]

# may be we need to position each entity in a graph to find distance/ time automatically
print(P,D,O,N_pickup, N_delivery, N, M)


[2, 3][5, 6][4][1, 2, 3, 4][4, 5, 6, 7][1, 2, 3, 4, 5, 6, 7]["v1", "v2"]

In [51]:
#****************************************
# parameters
#*****************************************
# p = number of pickup nodes
p = length(P)

# d = number of delivery nodes
d = length(D)

# o = number of cross dock
o = length(O)

# p_cd: number of nodes in pickup process (dummy CD + P + CD)
p_cd = length(N_pickup)

# d_cd: number of nodes in delivery process (CD + P + dummy CD)
d_cd = length(N_delivery)

# n: total nodes in the network
n = length(N)

# number of availble vehicles
m = length(M)

# p_i: quantity of products to be collected at pickup node i; [include product type in future]
p_i = [0, 50, 50]

# d_i: quantity of products to be delivered at delivery node i; [include product type in future]
d_i = [0, 0, 0, 0, 50, 50]

# capacity of vehicle (homogenous)
Q = 50

# Random.seed!(314)
# c_ij = the travel time (minutes)/cost between node i and node j; (assuming the cost is the same as the time req.)
t_ij_pick = 
    [0 20 30 0;
    20 0 50 40;
    30 50 0 40;
    0 40 40 0]

t_ij_del = 
    [0 70 80 0;
    20 0 70 60;
    80 70 0 60;
    0 60 60 0]

# Cross-dock activity time
T_o = 15

# time horizon (16 hr a day)
T_max = 16*60

960

In [106]:
# ****************************************
# Create a JuMP model
# ****************************************
cd_modl = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [107]:
# ****************************************
# Decision variables
#*****************************************
# x_ijk = 1 if vehicle k travels from node i to node j; 0 otherwise;
@variable(cd_modl, x_ijk[i=1:n, j=1:n, k=1:m], Bin)

# y_i = total quantity of collected/delivered products after leaving node i
@variable(cd_modl, y_i[i = 1:n], Int)

# t_pickup = latest arrival time a cross dock
@variable(cd_modl, t_pick[pick = 1:o])

# AT_k = arrival time of vehicle k at the cross-dock
@variable(cd_modl, AT_k[k =1:m])

2-element Array{VariableRef,1}:
 AT_k[1]
 AT_k[2]

In [108]:
#*****************************************
# Constraints
#*****************************************

# Pick-up Process
#----------------


# [Pick-up Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl, p_node_entry[j= P],
                    sum(x_ijk[i,j,k] for i=vcat(O_pickup, P) for k=1:m if i != j) == 1)

#[Pick-up Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl, p_node_exit[i= P],
                    sum(x_ijk[i,j,k] for j=vcat(P,O) for k=1:m if i != j) == 1)

# [Pick-up Process] 03: Consecutive movement of vehicles
@constraint(cd_modl, cons_move[l = P, k=1:m],
    sum(x_ijk[i,l,k] for i=vcat(O_pickup, P) if i != l) 
    -sum(x_ijk[l,j,k] for j=vcat(P,O) if j != l) 
    == 0)

# [Pick-up Process] 04: vehicle leave at dummy cross dock
@constraint(cd_modl, p_dum_leave[k=1:m],
    sum(x_ijk[i,j,k] for i=O_pickup for j = P) == 1)


# [Pick-up Process] 05: vehicle that leave at dummy cross dock visit cross dock immdiately after last pickup
@constraint(cd_modl, p_cd_entry[k=1:m],
    sum(x_ijk[i,j,k] for i= P for j = O) == 1)

# [Pick-up Process] 06: 
@constraint(cd_modl, p_pick_qnt[i = P, j = P, k=1:m; i!=j],
    y_i[j] 
    - y_i[i] 
    - p_i[j] 
    + Q 
    - (Q*(x_ijk[i,j,k] + x_ijk[i,j,k])) 
    + ((p_i[j]+p_i[i])*x_ijk[i,j,k])
    >= 0)

# [Pick-up Process] 07: 
@constraint(cd_modl, p_pick_qnt2[i = O_pickup, j = P, k=1:m],
    y_i[j]
    - x_ijk[i,j,k]*p_i[j]
    - (1-x_ijk[i,j,k])*Q
    <= 0)

# [Pick-up Process] 08a: 
@constraint(cd_modl, pick_cap1[i = P],
    y_i[i] >= p_i[i] )

# [Pick-up Process] 08b: 
@constraint(cd_modl, pick_cap2[i = P],
    y_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [2, 3]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 pick_cap2[2] : y_i[2] <= 50.0
 pick_cap2[3] : y_i[3] <= 50.0

In [109]:
#*****************************************
# Constraints
#*****************************************

# Delivery Process
#----------------

# [Delivery Process] 01: only one vehicle can arrive at a pickup node
@constraint(cd_modl, d_node_entry[j= D],
                    sum(x_ijk[i,j,k] for i=vcat(O, D) for k=1:m if i != j) == 1)

# [Delivery Process] 02: Only one vehicle depart from the pickup node
@constraint(cd_modl, d_node_exit[i= D],
                    sum(x_ijk[i,j,k] for j=vcat(D,O_delivery) for k=1:m if i != j) == 1)

# [Delivery Process] 03: Consecutive movement of vehicles
@constraint(cd_modl, cons_move_del[l = D, k=1:m],
    sum(x_ijk[i,l,k] for i=vcat(O, D) if i != l) 
    -sum(x_ijk[l,j,k] for j=vcat(D,O_delivery) if j != l) 
    == 0)

# [Delivery Process] 04: vehicle enter at dummy cross dock after last delivery
@constraint(cd_modl, d_dum_enter[k=1:m],
    sum(x_ijk[i,j,k] for i= D for j = O_delivery) == 1)

# [Delivery Process] 05: vehicle leave cross dock
@constraint(cd_modl, d_cd_leave[k=1:m],
    sum(x_ijk[i,j,k] for i= O for j = D) == 1)

# [Delivery Process] 06: 
@constraint(cd_modl, d_del_qnt[i = D, j = D, k=1:m; i!=j],
    y_i[j] 
    - y_i[i] 
    - d_i[j] 
    + Q 
    - (Q*(x_ijk[i,j,k] + x_ijk[i,j,k])) 
    + ((d_i[j]+d_i[i])*x_ijk[i,j,k])
    >= 0)

# [Delivery Process] 07: 
@constraint(cd_modl, d_del_qnt2[i = O, j = D, k=1:m],
    y_i[j]
    - x_ijk[i,j,k]*d_i[j]
    - (1-x_ijk[i,j,k])*Q
    <= 0)

# [Delivery Process] 08a: 
@constraint(cd_modl, del_cap1[i = D],
    y_i[i] >= d_i[i] )

# [Delivery Process] 08b: 
@constraint(cd_modl, del_cap2[i = D],
    y_i[i] <= Q)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [5, 6]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 del_cap2[5] : y_i[5] <= 50.0
 del_cap2[6] : y_i[6] <= 50.0

In [110]:
#*****************************************
# Constraints
#*****************************************

# @cross dock:: Connection Between Pickup and Delivery Process (using time)
#------------------------------------------------------------

# [cross-dock] 01: track time for product collection in the pickup process
@constraint(cd_modl, cd_prod_col[k = 1: m],
    AT_k[k]
    -sum(t_ij_pick[i,j] * x_ijk[i,j,k] for i=vcat(O_pickup, P) for j=vcat(P, O) if i != j) >= 0)

# [cross-dock] 02: all trucks arrive at the same time at cross-dock
@constraint(cd_modl, cd_truck_arr[k = 1: m, k_prime = 1:m; k != k_prime],
    AT_k[k] - AT_k[k_prime] == 0)

# [cross-dock] 03: total time at the pickup process
@constraint(cd_modl, time_pick[k = 1: m],
    t_pick[o] - AT_k[k] >= 0)

# [cross-dock] 04: total transportation time and processing time does not exceed the planning horizon T_max
@constraint(cd_modl, time_hor[k = 1: m],
    t_pick[o]+ T_o + sum(t_ij_del[i,j] * x_ijk[i,j,k] for i = vcat(O, D) - [3,3,3] for j = vcat(D, O_delivery)- [3,3,3]) - T_max <= 0)


2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 time_hor[1] : 70 x_ijk[1,2,1] + 70 x_ijk[3,2,1] + 80 x_ijk[1,3,1] + 70 x_ijk[2,3,1] + 60 x_ijk[2,4,1] + 60 x_ijk[3,4,1] + t_pick[1] <= 945.0
 time_hor[2] : 70 x_ijk[1,2,2] + 70 x_ijk[3,2,2] + 80 x_ijk[1,3,2] + 70 x_ijk[2,3,2] + 60 x_ijk[2,4,2] + 60 x_ijk[3,4,2] + t_pick[1] <= 945.0

In [117]:
#*****************************************
# Objective
#*****************************************

@objective(cd_modl, Min, sum(t_ij_pick[i,j]*x_ijk[i,j,k] for i=vcat(O_pickup, P) for j=vcat(P, O) for k=1:m) + sum(t_ij_del[i,j]*x_ijk[i,j,k] for i=vcat(O, D) - [3,3,3] for j=vcat(D, O_delivery)-[3,3,3] for k=1:m))

90 x_ijk[1,2,1] + 90 x_ijk[1,2,2] + 110 x_ijk[1,3,1] + 110 x_ijk[1,3,2] + 120 x_ijk[2,3,1] + 120 x_ijk[2,3,2] + 100 x_ijk[2,4,1] + 100 x_ijk[2,4,2] + 120 x_ijk[3,2,1] + 120 x_ijk[3,2,2] + 100 x_ijk[3,4,1] + 100 x_ijk[3,4,2]

In [86]:
print(cd_modl)

Min 40 x_ijk[1,2,1] + 40 x_ijk[1,2,2] + 60 x_ijk[1,3,1] + 60 x_ijk[1,3,2] + 100 x_ijk[2,3,1] + 100 x_ijk[2,3,2] + 80 x_ijk[2,4,1] + 80 x_ijk[2,4,2] + 100 x_ijk[3,2,1] + 100 x_ijk[3,2,2] + 80 x_ijk[3,4,1] + 80 x_ijk[3,4,2]
Subject to
 p_node_entry[2] : x_ijk[1,2,1] + x_ijk[3,2,1] + x_ijk[1,2,2] + x_ijk[3,2,2] == 1.0
 p_node_entry[3] : x_ijk[1,3,1] + x_ijk[2,3,1] + x_ijk[1,3,2] + x_ijk[2,3,2] == 1.0
 p_node_exit[2] : x_ijk[2,3,1] + x_ijk[2,4,1] + x_ijk[2,3,2] + x_ijk[2,4,2] == 1.0
 p_node_exit[3] : x_ijk[3,2,1] + x_ijk[3,4,1] + x_ijk[3,2,2] + x_ijk[3,4,2] == 1.0
 cons_move[2,1] : x_ijk[1,2,1] + x_ijk[3,2,1] - x_ijk[2,3,1] - x_ijk[2,4,1] == 0.0
 cons_move[3,1] : -x_ijk[3,2,1] + x_ijk[1,3,1] + x_ijk[2,3,1] - x_ijk[3,4,1] == 0.0
 cons_move[2,2] : x_ijk[1,2,2] + x_ijk[3,2,2] - x_ijk[2,3,2] - x_ijk[2,4,2] == 0.0
 cons_move[3,2] : -x_ijk[3,2,2] + x_ijk[1,3,2] + x_ijk[2,3,2] - x_ijk[3,4,2] == 0.0
 d_node_entry[5] : x_ijk[4,5,1] + x_ijk[6,5,1] + x_ijk[4,5,2] + x_ijk[6,5,2] == 1.0
 d_node_entry[6

In [118]:
optimize!(cd_modl)

In [119]:
@show objective_value(cd_modl)

objective_value(cd_modl) = 400.0


400.0

In [98]:
@show value.(x_ijk);

value.(x_ijk) = [0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [120]:
@show value.(y_i);

value.(y_i) = [0.0, 50.0, 50.0, 0.0, 50.0, 50.0, 0.0]


In [69]:
@show value.(AT_k);

value.(AT_k) = [99.99999999999999, 99.99999999999999]


In [70]:
@show value.(t_pick);

value.(t_pick) = [99.99999999999999]


In [116]:
for k=1:m
    print("\n veh: ", k, "\t")
    for i=1:n
        for j=1:n
            if value.(x_ijk[i,j,k]) == 1
                print(" ; ", i, "-->", j)
            end
        end
    end
end


 veh: 1	 ; 1-->2 ; 2-->4 ; 4-->6 ; 6-->7
 veh: 2	 ; 1-->3 ; 3-->4 ; 4-->5 ; 5-->7